In [ ]:
import os
import sys
import traceback
import csv
from datetime import datetime, timezone
from collections import deque
import threading
import time

import dash
from dash import dcc, html, dash_table, Output, Input, State, no_update
import plotly.graph_objs as go
import plotly.io as pio
import pandas as pd

from arduino_iot_cloud import ArduinoCloudClient

DEVICE_ID  = "f9be03ce-1809-4bf5-93c1-b3a4cc67a3e4"
SECRET_KEY = "7L4y@Ol4!jggkGIFrk2pKONxL"  # keep as-is if correct

VAR_X = "py_x"
VAR_Y = "py_y"
VAR_Z = "py_z"

DEFAULT_N = 10           # samples per plotted/saved window (change in UI)
RX_MAX    = DEFAULT_N * 5  # receive buffer capacity
OUT_DIR   = "out"          # folder for saved CSV/PNG/HTML

rx_lock   = threading.Lock()
rx_buffer = deque(maxlen=RX_MAX)  # holds (iso_time, x, y, z)

latest = {"x": None, "y": None, "z": None}
seen   = {"x": False, "y": False, "z": False}

def iso_now():
    return datetime.now(timezone.utc).astimezone().isoformat(timespec="milliseconds")

def handle_axis(axis_key, value):
    try:
        latest[axis_key] = float(value)
    except (TypeError, ValueError):
        return
    seen[axis_key] = True
    if all(seen.values()):
        with rx_lock:
            rx_buffer.append((iso_now(), latest["x"], latest["y"], latest["z"]))
        for k in seen:
            seen[k] = False

def on_x_changed(client, value): handle_axis("x", value)
def on_y_changed(client, value): handle_axis("y", value)
def on_z_changed(client, value): handle_axis("z", value)

def cloud_thread():
    global client
    try:
        client = ArduinoCloudClient(device_id=DEVICE_ID,
                                    username=DEVICE_ID,
                                    password=SECRET_KEY)
        client.register(VAR_X, value=None, on_write=on_x_changed)
        client.register(VAR_Y, value=None, on_write=on_y_changed)
        client.register(VAR_Z, value=None, on_write=on_z_changed)

        client.start()
        print("[cloud] Client started. Waiting for phone data (py_x/py_y/py_z)...")
    except Exception as e:
        print("[cloud] ERROR:", e)
        traceback.print_exc()
        while True:
            time.sleep(5)

def pop_exactly_n(n):
    rows = []
    with rx_lock:
        if len(rx_buffer) >= n:
            for _ in range(n):
                rows.append(rx_buffer.popleft())
    if not rows:
        return None
    return pd.DataFrame(rows, columns=["time", "x", "y", "z"])

def build_fig(df: pd.DataFrame, subtitle: str):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df["time"], y=df["x"], mode="lines", name="x"))
    fig.add_trace(go.Scatter(x=df["time"], y=df["y"], mode="lines", name="y"))
    fig.add_trace(go.Scatter(x=df["time"], y=df["z"], mode="lines", name="z"))
    fig.update_layout(
        title=f"Smartphone Accelerometer — {subtitle}",
        xaxis_title="Time",
        yaxis_title="Acceleration (device units)",
        margin=dict(l=40, r=20, t=60, b=40),
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    )
    return fig

def compute_stats(df: pd.DataFrame):
    rows = []
    for axis in ["x", "y", "z"]:
        s = pd.to_numeric(df[axis], errors="coerce").dropna()
        rows.append({
            "axis": axis,
            "mean": float(s.mean()),
            "std": float(s.std()),
            "rms": float((s.pow(2).mean()) ** 0.5),
            "min": float(s.min()),
            "max": float(s.max()),
            "peak_to_peak": float(s.max() - s.min()),
        })
    corr = df[["x", "y", "z"]].corr().round(3)
    return pd.DataFrame(rows), corr

def save_window(df: pd.DataFrame, fig, activity_label: str):
    os.makedirs(OUT_DIR, exist_ok=True)
    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    label = (activity_label or "unlabeled").strip().replace(" ", "-")[:24] or "unlabeled"
    csv_path  = os.path.join(OUT_DIR, f"window_{ts}_{label}.csv")
    png_path  = os.path.join(OUT_DIR, f"plot_{ts}_{label}.png")
    html_path = os.path.join(OUT_DIR, f"plot_{ts}_{label}.html")
    df.to_csv(csv_path, index=False)
    try:
        pio.write_image(fig, png_path, scale=2)  # needs `pip install kaleido`
    except Exception as e:
        print(f"[warn] PNG export failed ({e}). HTML will still be saved.")
    fig.write_html(html_path, include_plotlyjs="cdn")
    return csv_path, png_path, html_path

app = dash.Dash(__name__)
app.title = "Accelerometer Live Viewer"

app.layout = html.Div(style={"maxWidth": "1100px", "margin": "0 auto"}, children=[
    html.H2("Smartphone Accelerometer → Arduino Cloud → Python → Dash"),
    html.Div([
        html.Div([
            html.Label("Window size N (samples):"),
            dcc.Input(id="win-n", type="number", min=50, step=50, value=DEFAULT_N),
        ], style={"display": "inline-block", "marginRight": "20px"}),
        html.Div([
            html.Label("Activity label:"),
            dcc.Input(id="activity", type="text", value="still", placeholder="e.g., still / walk / shake"),
        ], style={"display": "inline-block", "marginRight": "20px"}),
        html.Div([
            html.Button("Force save next N samples", id="force-btn"),
            html.Span(id="save-msg", style={"marginLeft": "12px"})
        ], style={"display": "inline-block"}),
    ], style={"marginBottom": "10px"}),
    dcc.Graph(id="live-graph", figure=go.Figure()),
    html.Div(id="stats-area"),
    dcc.Interval(id="tick", interval=1000, n_intervals=0)  # check buffer every second
])

@app.callback(
    Output("live-graph", "figure"),
    Output("stats-area", "children"),
    Output("save-msg", "children"),
    Input("tick", "n_intervals"),
    Input("force-btn", "n_clicks"),
    State("win-n", "value"),
    State("activity", "value"),
    State("live-graph", "figure"),
)
def update_graph(_n_intervals, _n_clicks, n, activity_label, current_fig):
    n = int(n or DEFAULT_N)
    df = pop_exactly_n(n)

    # If not enough samples yet, keep the current UI unchanged
    if df is None:
        return no_update, no_update, ""

    subtitle = f"{len(df)} samples — activity='{activity_label or 'unlabeled'}'"
    fig = build_fig(df, subtitle)
    stats_df, corr_df = compute_stats(df)

    csv_path, png_path, html_path = save_window(df, fig, activity_label)
    save_msg = f"Saved: {os.path.basename(csv_path)}, {os.path.basename(png_path)}, {os.path.basename(html_path)}"

    stats_table = dash_table.DataTable(
        data=stats_df.round(4).to_dict("records"),
        columns=[{"name": c, "id": c} for c in stats_df.columns],
        style_table={"overflowX": "auto"},
        style_cell={"padding": "6px"},
    )
    corr_table = dash_table.DataTable(
        data=corr_df.reset_index().rename(columns={"index": "axis"}).to_dict("records"),
        columns=[{"name": c, "id": c} for c in ["axis", "x", "y", "z"]],
        style_table={"overflowX": "auto", "marginTop": "8px"},
        style_cell={"padding": "6px"},
    )
    stats_block = html.Div([
        html.H4("Window statistics"), stats_table,
        html.Hr(), html.H4("Axis correlation"), corr_table
    ])

    return fig, stats_block, save_msg

if __name__ == "__main__":
    t = threading.Thread(target=cloud_thread, daemon=True)
    t.start()

    # Run Dash app
    app.run(host="127.0.0.1", port=8050, debug=False, jupyter_mode= "tab")

Dash app running on http://127.0.0.1:8050/


<IPython.core.display.Javascript object>

ERROR:root:task: connection_task raised exception: .
ERROR:root:task: discovery raised exception: .


[warn] PNG export failed (
Image export using the "kaleido" engine requires the Kaleido package,
which can be installed using pip:

    $ pip install --upgrade kaleido
). HTML will still be saved.
[warn] PNG export failed (
Image export using the "kaleido" engine requires the Kaleido package,
which can be installed using pip:

    $ pip install --upgrade kaleido
). HTML will still be saved.
[warn] PNG export failed (
Image export using the "kaleido" engine requires the Kaleido package,
which can be installed using pip:

    $ pip install --upgrade kaleido
). HTML will still be saved.
[warn] PNG export failed (
Image export using the "kaleido" engine requires the Kaleido package,
which can be installed using pip:

    $ pip install --upgrade kaleido
). HTML will still be saved.
[warn] PNG export failed (
Image export using the "kaleido" engine requires the Kaleido package,
which can be installed using pip:

    $ pip install --upgrade kaleido
). HTML will still be saved.
[warn] PNG expo